In [ ]:
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(1)

In [ ]:
import keras
from keras import backend as K
from keras import applications
from keras import Model
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Flatten, BatchNormalization, GlobalMaxPooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, Activation
from keras.models import load_model
import matplotlib.pyplot as plt
import numpy as np
from keras.callbacks import History 
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
import warnings

# deal with potential warnings
warnings.filterwarnings(action='ignore')

In [ ]:
(x_train, y_train, x_dev, y_dev) = np.load('data/save_data.npy')
x_test = np.load('data/testdata.npy')
testID = np.load('data/testdataID.npy')

x_test /= 255

load pre-trained model

In [ ]:
model = applications.VGG16(weights = "imagenet", include_top=False, input_shape = (64, 64, 3))

In [ ]:
model.summary()
print(len(model.layers))

In [ ]:
# callbacks, early-stopping
callbacks = keras.callbacks.ModelCheckpoint('transfer_1.h5', 
                                             monitor='val_acc', verbose=1, save_best_only=True, 
                                             save_weights_only=False, mode='auto', period=1)

callback_list = [callbacks]

In [ ]:
for layer in model.layers[:-4]:
    layer.trainable = False

#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(512, activation="relu")(x)
predictions = Dense(1, activation="sigmoid")(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)

# compile the model 
model_final.compile(loss = "binary_crossentropy", optimizer = keras.optimizers.Adam(lr=1e-4, amsgrad=True), metrics=["accuracy"])

# Initiate the train and test generators with data Augumentation 
datagen = ImageDataGenerator(horizontal_flip = True,
                             zoom_range = 0.2,
                             width_shift_range = 0.1,
                             height_shift_range=0.1,
                             rotation_range=20)

In [ ]:
# training
# temp_weights = [layer.get_weights() for layer in new_model.layers]

# for j in range(len(temp_weights)):
#     model_final.layers[j].set_weights(temp_weights[j])
    
model_final.fit_generator(
    datagen.flow(x_train, y_train, batch_size=32), 
    verbose=1,
    steps_per_epoch=len(x_train) // 32,
    validation_data=(x_dev,y_dev), 
    epochs=150,
    callbacks=callback_list)

In [ ]:
new_model = load_model('transfer_1.h5')
new_model.summary()

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score

y_pre = np.round(new_model.predict(x_dev, verbose=1))
confusion_matrix(y_dev, y_pre)

submission

In [ ]:
# predict on test set 
score = np.round(new_model.predict(x_test, verbose=1)).astype(np.int32)

In [ ]:
sub = pd.DataFrame()
sub['id'] = pd.Series(testID)
sub['label'] = score
sub.to_csv('DvCsubmission.csv', index = False)